In [28]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, asin, sqrt, pi, atan, atan2, fabs
import time

In [2]:
def calcular_distancia(lat1,long1,lat2,long2):
    lon1, lat1 = (radians(coord) for coord in (long1,lat1))
    lon2, lat2 = (radians(coord) for coord in (long2,lat2))
    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    a = (
        sin(dlat * 0.5)**2 +
        cos(lat1) * cos(lat2) * sin(dlon * 0.5)**2
    )
    
    radioTierra = 6371008.8
    return 2 * radioTierra * asin(sqrt(a)) / 1000

#distancia de Haversine
#https://gist.github.com/habibutsu/8bbcc202a915e965c6a6d4f561d0e482

In [3]:
propiedades = pd.read_csv('/home/mati/Desktop/properati.csv')
#propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/properati.csv')

In [4]:
propiedades = propiedades.loc[propiedades.Year >= 2016,:]

In [5]:
propiedades = propiedades.loc[(propiedades.lat.notnull()) & (propiedades.lon.notnull()) &\
                             (propiedades.superficie.notnull()) & (propiedades.price_aprox_usd.notnull()),:]

## Tomo como set de prueba las publicaciones de junio 2017

In [36]:
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),:].reset_index()
set_datos = propiedades.loc[(propiedades.Year != 2017) | (propiedades.Month < 6),:].reset_index()

In [37]:
set_pruebas = set_pruebas.sample(frac = 0.12).reset_index()
set_pruebas.loc[:,'resultado'] = 0
set_pruebas.loc[:,'cantidad_analizados'] = 0
set_pruebas = set_pruebas.loc[:,['property_type','lat','lon','superficie','price_aprox_usd','resultado','cantidad_analizados']]
set_pruebas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7519 entries, 0 to 7518
Data columns (total 7 columns):
property_type          7519 non-null object
lat                    7519 non-null float64
lon                    7519 non-null float64
superficie             7519 non-null float64
price_aprox_usd        7519 non-null float64
resultado              7519 non-null int64
cantidad_analizados    7519 non-null int64
dtypes: float64(4), int64(2), object(1)
memory usage: 411.3+ KB


In [38]:
set_datos = set_datos.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464833 entries, 0 to 464832
Data columns (total 5 columns):
property_type      464833 non-null object
lat                464833 non-null float64
lon                464833 non-null float64
superficie         464833 non-null float64
price_aprox_usd    464833 non-null float64
dtypes: float64(4), object(1)
memory usage: 17.7+ MB


### Agrupo por tipo de propiedad y luego por superficie, serian los lideres de knn

In [39]:
def redondear_superficie(superficie):
    """Convierto la superficie a un multiplo de 10"""
    superficie /= 10
    superficie = round(superficie) #lo hago entero
    superficie *= 10
    return int(superficie)

In [40]:
set_datos.loc[:,'superficie_actualizada'] = set_datos.loc[:,'superficie'].apply(redondear_superficie)
set_datos.head(2)

,property_type,lat,lon,superficie,price_aprox_usd,superficie_actualizada
0,house,-34.603282,-58.525425,448.0,490000.0,450
1,apartment,-34.564741,-58.441372,308.0,810000.0,310


In [41]:
set_datos_agrupados = set_datos.groupby('property_type')

In [42]:
#Agrupo por superficie cada grupo
grupos = {}
for grupo in set_datos_agrupados.__iter__():
    clave = grupo[0]
    valor = grupo[1].groupby('superficie_actualizada')
    diccionario = {}
    for group in valor.__iter__():
        sub_clave = group[0]
        sub_valor = group[1]
        diccionario[sub_clave] = sub_valor
    grupos[clave] = diccionario

In [43]:
set_datos_agrupados = grupos

#### Los datos quedan dic = {prop_type: {sup : dataframe}}

## Algoritmo de KNN

In [48]:
def calcular_distancias_para_knn(datos,prueba_type,prueba_sup,prueba_lat,prueba_lon):
    grupo = datos[prueba_type]
    prueba_sup = redondear_superficie(prueba_sup)
    analizados = []
    for sup in range(prueba_sup - 10, prueba_sup + 20, 10):  #Miro sup +- 10
        try:
            a_analizar = grupo[sup].reset_index()
            for i in range(len(a_analizar)):
                distancia = calcular_distancia(a_analizar.lat[i],a_analizar.lon[i],prueba_lat,prueba_lon)
                analizados.append((distancia,a_analizar.price_aprox_usd[i]))
        except:
            #Si no hay ninguna propiedad con superficie sup
            continue
    return analizados

In [71]:
def knn(datos,pruebas,k):
    for i in range(len(pruebas)):
        distancias = calcular_distancias_para_knn(datos,pruebas.property_type[i],pruebas.superficie[i],\
                                                  pruebas.lat[i],pruebas.lon[i])
        resultado = obtener_promedio_k_minimos(distancias,k)
        pruebas.resultado[i] = resultado
        pruebas.cantidad_analizados[i] = len(distancias)
    return pruebas

In [63]:
def obtener_promedio_k_minimos(distancias,k):
    distancias.sort()
    suma = 0.0
    k = min(k,len(distancias))
    for x in range(k):
        suma += distancias[x][1]
    return suma / k

In [50]:
def calcular_precision(resultados):
    res = resultados.loc[:,['price_aprox_usd','resultado']].apply(lambda x: calcular_resultado(x[0],x[1]),axis = 1)
    return round(sum(res)*100 / len(res),2)

In [67]:
def calcular_resultado(real,calculado):
    porcentaje = 0.025
    
    if abs(real - calculado) <= real * porcentaje:
        return 1
    return 0

## Busqueda del hiperparametro K

##### Empezamos con algunos k salteados

In [68]:
lista_k = [10,50,100,200,350,500,750,1000,2000,3000,10000,20000]

for k in lista_k:
    print("inicio: ",time.strftime("%H:%M:%S"))
    pruebas = knn(set_datos_agrupados,set_pruebas,k)
    precision = calcular_precision(pruebas)
    print("Para k = {} la precision es {}%".format(k,precision))
    print("final: ",time.strftime("%H:%M:%S"))


inicio:  10:35:52
0


/home/mati/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/mati/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/mati/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


1
2
3
4
5
6
7
8
9
final:  10:39:43


,property_type,lat,lon,superficie,price_aprox_usd,resultado,cantidad_analizados
0,apartment,-34.541879,-58.476834,70.0,175500.0,199338,78156
1,PH,-34.575951,-58.537270,55.0,68000.0,62163,9627
2,apartment,-34.591299,-58.395343,38.0,142205.0,154851,134671
3,apartment,-34.604807,-58.363740,120.0,640000.0,693755,15617
4,apartment,-34.644354,-58.591975,54.0,130000.0,96653,139872
5,apartment,-34.608163,-58.558240,42.0,81000.0,77142,134671
6,house,-34.346581,-58.860268,218.0,295000.0,290183,10974
7,house,-34.395002,-58.738442,80.0,105000.0,100183,10490
8,apartment,-34.643861,-58.509787,52.0,140000.0,84142,139872
9,apartment,-34.628247,-58.484411,60.0,80000.0,102363,96337
